**Dataset details**
CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.


In [0]:
# Load necessary packages
from keras.datasets import cifar10
from keras.utils import np_utils
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

In [0]:
# load the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [11]:
# Lets determine the dataset characteristics
print('Training Images: {}'.format(X_train.shape))
print('Testing Images: {}'.format(X_test.shape))

Training Images: (50000, 32, 32, 3)
Testing Images: (10000, 32, 32, 3)


### 2. Preprocessing the dataset

First things first, we need to preprocess the dataset so the images and labels are in a form that Keras can ingest. To start, we'll define a NumPy seed for reproducibility, then normalize the images. 

Furthermore, we will also convert our class labels to one-hot vectors.  This is a standard output format for neural networks. 

In [0]:
np.random.seed(6) 

# normalize the inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [13]:
# class labels shape
print(y_train.shape)
print(y_train[0])

(50000, 1)
[6]


**One Hot Encoding**
In one hot encoding, the class label of 6 should be denoted [0, 0, 0, 0, 0, 0, 1, 0, 0, 0].  We can accomplish this using the np_utils.to_categorical() function.

In [14]:
# hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

print(y_train.shape)
print(y_train[0])

(50000, 10)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [15]:
print(X_train.shape)

(50000, 32, 32, 3)


**Generating model**

In [20]:
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, GlobalAveragePooling2D
from tensorflow.python.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

#change the path to your directory
where_to_save= '/content/sample_data/weights.hdf5'
monitor='val_accuracy'

#These hyperparameters are used to save time
epochs = 5
batch_size = 100
#Below hyperparamets will provide more accuracy
# epochs = 350
# batch_size = 32

#Define model architecture
model = Sequential()

model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(32, 32, 3)))
model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(96, (3, 3), padding = 'same', strides = (2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(192, (3, 3), padding = 'same', strides = (2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
model.add(Conv2D(192, (1, 1), activation='relu', padding = 'valid'))
model.add(Conv2D(10, (1, 1), padding = 'valid'))

# add GlobalAveragePooling2D layer with Softmax activation
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy', 
    optimizer=sgd, 
    metrics=['accuracy'],
            )

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, 
          verbose = 1,callbacks=[
                                EarlyStopping(verbose=True, patience=5, monitor=monitor),
                                ModelCheckpoint(where_to_save, monitor=monitor, verbose=True,save_best_only=True)
                                ]
    )


Epoch 1/5
500/500 [==============================] - ETA: 0s - loss: 2.1217 - accuracy: 0.1938
Epoch 00001: val_accuracy improved from -inf to 0.29220, saving model to /content/sample_data/weights.hdf5
500/500 [==============================] - 26s 53ms/step - loss: 2.1217 - accuracy: 0.1938 - val_loss: 1.8889 - val_accuracy: 0.2922
Epoch 2/5
500/500 [==============================] - ETA: 0s - loss: 1.8054 - accuracy: 0.3291
Epoch 00002: val_accuracy improved from 0.29220 to 0.38880, saving model to /content/sample_data/weights.hdf5
500/500 [==============================] - 27s 53ms/step - loss: 1.8054 - accuracy: 0.3291 - val_loss: 1.6356 - val_accuracy: 0.3888
Epoch 3/5
500/500 [==============================] - ETA: 0s - loss: 1.5848 - accuracy: 0.4136
Epoch 00003: val_accuracy improved from 0.38880 to 0.46540, saving model to /content/sample_data/weights.hdf5
500/500 [==============================] - 27s 53ms/step - loss: 1.5848 - accuracy: 0.4136 - val_loss: 1.4394 - val_accura

In [21]:
# print model summary
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_8 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 16, 16, 192)       331968    
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 8, 8, 192)        

In [23]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

313/313 [==============================] - 2s 8ms/step - loss: 1.2102 - accuracy: 0.5655
Accuracy: 56.55%
